In [2]:
import numpy as np

In [6]:
n_m = 400
a_m = 1
a_c = 1
q = a_c / a_m
phi_m = 0.003
y_m = phi_m / (1-phi_m)
a_spt = (1+q)**3 - 1
b_spt = 9 * q**2 /2 + 3 * q**3
c_spt = 3 * q**3
Q_spt = a_spt * y_m + b_spt * y_m**2 + c_spt * y_m**3
alpha_spt = (1-phi_m) * np.exp(-1 * Q_spt)
print(alpha_spt)

0.9761532493169941
